In [ ]:
# Check/import environment to control notebook from CI
ENV = %env
try:
    ENV['BRANCH']
    branch=ENV['BRANCH']
except:
    branch="master"

In [ ]:
# Continuous Benchmarking Results for PEPC

In [ ]:
from IPython.display import Markdown as md
md("# Branch: %s"%(branch))

Plots are interactive and can be zoomed, shifted, and data shown can be selected (via the legend). Double click to reset.

In [ ]:
import sqlite3

import pandas as pd
import plotly.express as px

# Define config options for plotly
px_config = dict({'scrollZoom': True, "displaylogo": False,'modeBarButtonsToRemove': ['lasso2d']}) # enable scroll/finger-zoom, hide plotly logo
# Check environment to decide on renderer

try:
    ENV['RENDER_CI']
    px_renderer = "notebook_connected" # "notebook_connected" will use CDNs for .js files instead of including them, this will break offline use
except:
    px_renderer = "jupyterlab" # render for the interactive jupyterlab

In [ ]:
# Create connection to database to read JUBE data
cnx = sqlite3.connect("../"+branch+"/result_database.sqlite")
# Read date into Pandas dataframe
df = pd.read_sql_query("SELECT * FROM full_results", cnx)
# Display dataframe
#df
# Find/display all columns contained in dataframe
#for col in df.columns:
#   print(col)

In [ ]:
# Rename some columns to be 'prettier'
df.rename(
    columns={
        "walk": "Tree Walk",
        "nodes": "Nodes",
        "chksum": "commit",
        "threads": "Threads",
        "particles": "Particles",
        "list_of_mpis": "Toolchain",
        "systemname": "System",
        "jube_benchmark_padid": "Id",
        "taskspnode": "Tasks per node",
        "part_per_thrd": "Particles per thread",
        "wallclock_cnt": "Iterations",
        "timesteps": "Steps",
        "tree_walk_cnt": "Timesteps",
        "wallclock_min": "Wallclock [s] (min)",
        "wallclock_avg": "Wallclock [s]",
        "wallclock_max": "Wallclock [s] (max)",
        "tree_walk_min": "Tree Walk [s] (min)",
        "tree_walk_avg": "Tree Walk [s]",
        "tree_walk_max": "Tree Walk [s] (max)",
        "tree_grow_min": "Tree Grow [s] (min)",
        "tree_grow_avg": "Tree Grow [s]",
        "tree_grow_max": "Tree Grow [s] (max)",
        "comm_recv_min": "Comm Receive [s] (min)",
        "comm_recv_avg": "Comm Receive [s]",
        "comm_recv_max": "Comm Receive [s] (max)",
        "comm_reqs_min": "Comm Request [s] (min)",
        "comm_reqs_avg": "Comm Request [s]",
        "comm_reqs_max": "Comm Request [s] (max)",
        "step_time_min": "Step Time [s] (min)",
        "step_time_avg": "Step Time [s]",
        "step_time_max": "Step Time [s] (max)",
    },
    inplace=True,
)

In [ ]:
# Convert '--all --long' checksum to only the SHA hash
df["commit"]=df["commit"].str.replace('.*-g','',regex=True)

## Overall timings

The solid red line represents a trend in form of a sliding average of 3 values.

In [ ]:
# Plot all benchmarks and systems in a grid

# Get number of result sets to scale size of plot
height = 0
for System in df["System"].unique():
#    height = max(height, len(df[df["System"] == System]["benchmark"].unique()) * 400)
    fig = px.scatter(
        df,
        x="Id",
        y="Wallclock [s]",
        #facet_col="System",
        marginal_y="rug",
        title=System.upper(),
        trendline="rolling",
        trendline_options=dict(window=3),
        trendline_color_override="red",
        #height=height,
    )
    fig.show(config=px_config)#, renderer=px_renderer)

### Statistics from all runs

In [ ]:
# Display stats for selected columns
for System in df["System"].unique():
    print("System: " + System.upper())
    display(
        df[(df["System"] == System)][
            [
                "Wallclock [s]",
                "Step Time [s]",
                "Tree Walk [s]",
                "Tree Grow [s]",
                "Comm Receive [s]",
                "Comm Request [s]",
            ]
        ].describe()
    )

## Results for each individual timer for each system

In [ ]:
# Massage our jube results to combine all timers
df_melt = df.melt(
    id_vars=[
        "Id",
        "commit",
        "System",
        "Toolchain",
        "Nodes",
        "Tasks per node",
        "Threads",
        "total_threads",
        "Wallclock [s]",
    ],
    value_vars=[
        "Tree Walk [s]",
        "Tree Grow [s]",
        "Comm Receive [s]",
        "Comm Request [s]",
        "Step Time [s]",
    ],
    var_name="Timer",
    value_name="Time [s]",
)

In [ ]:
# Generate plots for each of the listed systems

for System in df_melt["System"].unique():
    ## Get number of result sets to scale size of plot
    #height = len(df_melt[df_melt["System"] == System]["benchmark"].unique()) * 400
    fig = px.scatter(
        df_melt[df_melt["System"] == System],
        x="commit",
        y="Time [s]",
        color="Timer",
        symbol="Timer",
        marginal_y="rug",
        #facet_row="benchmark",
        title=System.upper(),
        #height=height,
    )
    fig.show(config=px_config)#, renderer=px_renderer)